<b>Important:</b> Save prepared/modified datasets locally to <code>../data/prepared/</code>

<b>IMPORTANT++</b> Prevent data leakage by only applying frequency encoding and target encoding on THE TRAINING DATASET!!! Do it later


In [36]:
import sys
sys.path.insert(1, '../../../../utils')
from pandas import read_csv, DataFrame
from dslabs_functions import *


In [37]:
file_tag = "flight"
filename = "../../data/raw/Combined_Flights_2022_cleaned.csv"
data: DataFrame = read_csv(filename, na_values="")

# sampling below to make testing more efficient
data = data.sample(n=200_000, random_state=42).reset_index(drop=True)

In [38]:
# Create copy of data before encoding
data_enc = data.copy()

# First drop DestStateName and OriginStateName as they are obviously redundant
data_enc = data_enc.drop(columns=["DestStateName", "OriginStateName"])

In [39]:
vars_types: dict[str, list] = get_variable_types(data_enc)
symbolic: list[str] = vars_types["symbolic"]

for var in symbolic:
    unique_values = data_enc[var].nunique()
    print(f"Variable '{var}': {unique_values} unique values.")


Variable 'Airline': 21 unique values.
Variable 'Origin': 373 unique values.
Variable 'Dest': 374 unique values.
Variable 'Marketing_Airline_Network': 10 unique values.
Variable 'Operated_or_Branded_Code_Share_Partners': 14 unique values.
Variable 'IATA_Code_Marketing_Airline': 10 unique values.
Variable 'Operating_Airline': 21 unique values.
Variable 'IATA_Code_Operating_Airline': 21 unique values.
Variable 'Tail_Number': 5843 unique values.
Variable 'OriginCityName': 367 unique values.
Variable 'OriginState': 52 unique values.
Variable 'DestCityName': 368 unique values.
Variable 'DestState': 53 unique values.
Variable 'DepTimeBlk': 19 unique values.
Variable 'ArrTimeBlk': 19 unique values.


### Cyclic encoding of time blocks

In [40]:
# For time blocks, do cyclic encoding
timeblk_order: list[str] = [
    "0001-0559", "0600-0659", "0700-0759", "0800-0859", "0900-0959",
    "1000-1059", "1100-1159", "1200-1259", "1300-1359", "1400-1459",
    "1500-1559", "1600-1659", "1700-1759", "1800-1859", "1900-1959",
    "2000-2059", "2100-2159", "2200-2259", "2300-2359"
]

# Create mapping dictionaries
DepTimeBlk_values: dict[str, int] = {v: i for i, v in enumerate(timeblk_order)}
ArrTimeBlk_values: dict[str, int] = {v: i for i, v in enumerate(timeblk_order)}

# Apply the mappings
encoding: dict[str, dict] = {
    "DepTimeBlk": DepTimeBlk_values,
    "ArrTimeBlk": ArrTimeBlk_values,
}
data_enc = data_enc.replace(encoding)

encode_cyclic_variables(data_enc, ["DepTimeBlk", "ArrTimeBlk"])
data_enc.head()


,FlightDate,Airline,Origin,Dest,Cancelled,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,...,DestStateFips,DestWac,DepTimeBlk,CRSArrTime,ArrTimeBlk,DistanceGroup,DepTimeBlk_sin,DepTimeBlk_cos,ArrTimeBlk_sin,ArrTimeBlk_cos
0,2022-03-19,SkyWest Airlines Inc.,ASE,DEN,False,1831,62.0,125.0,2022,1,...,8,82,13,1933,14,1,-0.985,-0.174,-0.985,0.174
1,2022-02-16,SkyWest Airlines Inc.,TYS,DEN,False,1605,212.0,1162.0,2022,1,...,8,82,11,1737,12,5,-0.643,-0.766,-0.866,-0.500
2,2022-01-18,American Airlines Inc.,LAX,OGG,False,1719,330.0,2486.0,2022,1,...,15,2,12,2049,15,10,-0.866,-0.500,-0.866,0.500
3,2022-01-23,Southwest Airlines Co.,LAS,BWI,False,1515,260.0,2106.0,2022,1,...,24,35,10,2235,17,9,-0.342,-0.940,-0.342,0.940
4,2022-07-01,Delta Air Lines Inc.,DTW,RDU,False,715,98.0,501.0,2022,3,...,37,36,2,853,3,3,0.643,0.766,0.866,0.500


### One-Hot-encoding
Applied on variables with less than <i>30</i> number of unique values (low cardinality)

In [41]:
# Get symbolic variables with less than 30 unique values
symbolic_low_card: list[str] = [
    var for var in symbolic if data_enc[var].nunique() < 30
]

# Dummify low cardinality symbolic variables
data_enc = dummify(data_enc, symbolic_low_card)
data_enc.head()

# Find remaining symbolic variables
symbolic_remaining: list[str] = [
    var for var in symbolic if var not in symbolic_low_card
]

print("Remaining symbolic variables after dummification:", symbolic_remaining)



Remaining symbolic variables after dummification: ['Origin', 'Dest', 'Tail_Number', 'OriginCityName', 'OriginState', 'DestCityName', 'DestState']


### Hierarchical encoding of state name

In [42]:
# Print all unique values in OriginState and DestState
print("Unique values in 'OriginState':", data_enc["OriginState"].unique())
print("Unique values in 'DestState':", data_enc["DestState"].unique())

# Encode OriginState and DestState using hierarchical encoding based on state regions
state_region_mapping: dict[str, str] = {
    # Northeast
    "ME": "Northeast", "NH": "Northeast", "VT": "Northeast", "MA": "Northeast",
    "RI": "Northeast", "CT": "Northeast", "NY": "Northeast", "NJ": "Northeast",
    "PA": "Northeast",
    # Midwest
    "OH": "Midwest", "IN": "Midwest", "IL": "Midwest", "MI": "Midwest",
    "WI": "Midwest", "MN": "Midwest", "IA": "Midwest", "MO": "Midwest",
    "ND": "Midwest", "SD": "Midwest", "NE": "Midwest", "KS": "Midwest",
    # South
    "DE": "South", "MD": "South", "VA": "South", "WV": "South",
    "NC": "South",  "SC": "South",  "GA": "South",  "FL": "South",
    "KY": "South",  "TN": "South",  "MS": "South",  "AL": "South",
    "OK": "South",  "TX": "South",  "AR": "South",  "LA": "South",
    # West
    "ID": "West",  "MT": "West",  "WY": "West",  "NV":  "West",
    "UT":  "West",  "CO": "West", "AZ": "West", "NM": "West",
    "CA": "West", "OR": "West", "WA": "West",
    # Non-continental US
   "AK": "Non-continental US", "HI": "Non-continental US",
   "PR": "Non-continental US", "VI": "Non-continental US",
    # Unknown/Other
   "TT": "Unknown/Other"
}

data_enc["OriginState"] = data_enc["OriginState"].map(state_region_mapping)
data_enc["DestState"] = data_enc["DestState"].map(state_region_mapping)

data_enc.head()

Unique values in 'OriginState': ['CO' 'TN' 'CA' 'NV' 'MI' 'WA' 'AZ' 'VA' 'IL' 'GA' 'FL' 'UT' 'OR' 'TX'
 'NY' 'ME' 'NC' 'NJ' 'PR' 'MN' 'PA' 'ND' 'CT' 'OK' 'AR' 'NE' 'LA' 'OH'
 'HI' 'MT' 'MD' 'MO' 'WY' 'MA' 'IN' 'AL' 'NM' 'IA' 'WI' 'SC' 'ID' 'MS'
 'KY' 'RI' 'SD' 'WV' 'AK' 'VI' 'NH' 'KS' 'VT' 'TT']
Unique values in 'DestState': ['CO' 'HI' 'MD' 'NC' 'UT' 'CA' 'AZ' 'GA' 'MN' 'FL' 'VA' 'NV' 'TX' 'PA'
 'NY' 'NJ' 'MI' 'WA' 'SC' 'OR' 'MT' 'KY' 'MA' 'AK' 'IL' 'TN' 'ND' 'MO'
 'OK' 'PR' 'WI' 'LA' 'AR' 'IN' 'MS' 'NM' 'IA' 'ID' 'OH' 'WY' 'KS' 'SD'
 'VT' 'RI' 'NE' 'AL' 'WV' 'NH' 'ME' 'CT' 'VI' 'TT' 'DE']


,FlightDate,Origin,Dest,Cancelled,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,Month,...,ArrTimeBlk_9,ArrTimeBlk_10,ArrTimeBlk_11,ArrTimeBlk_12,ArrTimeBlk_13,ArrTimeBlk_14,ArrTimeBlk_15,ArrTimeBlk_16,ArrTimeBlk_17,ArrTimeBlk_18
0,2022-03-19,ASE,DEN,False,1831,62.0,125.0,2022,1,3,...,False,False,False,False,False,True,False,False,False,False
1,2022-02-16,TYS,DEN,False,1605,212.0,1162.0,2022,1,2,...,False,False,False,True,False,False,False,False,False,False
2,2022-01-18,LAX,OGG,False,1719,330.0,2486.0,2022,1,1,...,False,False,False,False,False,False,True,False,False,False
3,2022-01-23,LAS,BWI,False,1515,260.0,2106.0,2022,1,1,...,False,False,False,False,False,False,False,False,True,False
4,2022-07-01,DTW,RDU,False,715,98.0,501.0,2022,3,7,...,False,False,False,False,False,False,False,False,False,False


In [43]:
# Dummify OriginState and DestState
data_enc = dummify(data_enc, ["OriginState", "DestState"])
data_enc.head()

,FlightDate,Origin,Dest,Cancelled,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,Month,...,OriginState_Northeast,OriginState_South,OriginState_Unknown/Other,OriginState_West,DestState_Midwest,DestState_Non-continental US,DestState_Northeast,DestState_South,DestState_Unknown/Other,DestState_West
0,2022-03-19,ASE,DEN,False,1831,62.0,125.0,2022,1,3,...,False,False,False,True,False,False,False,False,False,True
1,2022-02-16,TYS,DEN,False,1605,212.0,1162.0,2022,1,2,...,False,True,False,False,False,False,False,False,False,True
2,2022-01-18,LAX,OGG,False,1719,330.0,2486.0,2022,1,1,...,False,False,False,True,False,True,False,False,False,False
3,2022-01-23,LAS,BWI,False,1515,260.0,2106.0,2022,1,1,...,False,False,False,True,False,False,False,True,False,False
4,2022-07-01,DTW,RDU,False,715,98.0,501.0,2022,3,7,...,False,False,False,False,False,False,False,True,False,False


In [44]:
# Check remaining symbolic variables
vars_types: dict[str, list] = get_variable_types(data_enc)
symbolic: list[str] = vars_types["symbolic"]

for var in symbolic:
    unique_values = data_enc[var].nunique()
    print(f"Variable '{var}': {unique_values} unique values.")

Variable 'Origin': 373 unique values.
Variable 'Dest': 374 unique values.
Variable 'Tail_Number': 5843 unique values.
Variable 'OriginCityName': 367 unique values.
Variable 'DestCityName': 368 unique values.


### Splitting Data before Frequency & Target Encoding

In [45]:
data_enc = data_enc.sort_values("FlightDate").reset_index(drop=True)

split_idx = int(0.7 * len(data_enc))
train_enc = data_enc.iloc[:split_idx].copy()
test_enc  = data_enc.iloc[split_idx:].copy()
print(f"Train size: {len(train_enc)}; Test size: {len(test_enc)}")

Train size: 140000; Test size: 60000


### Frequency encoding

In [46]:
for var in ["Dest", "Origin", "OriginCityName", "DestCityName"]:
    # Fit on train only
    freq_encoding = train_enc[var].value_counts(normalize=True).to_dict()
    # Apply to train and test with same mapping
    train_enc[var] = train_enc[var].map(freq_encoding)
    test_enc[var]  = test_enc[var].map(freq_encoding)

train_enc.head()

,FlightDate,Origin,Dest,Cancelled,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,Month,...,OriginState_Northeast,OriginState_South,OriginState_Unknown/Other,OriginState_West,DestState_Midwest,DestState_Non-continental US,DestState_Northeast,DestState_South,DestState_Unknown/Other,DestState_West
0,2022-01-01,0.007171,0.023493,False,1500,85.0,386.0,2022,1,1,...,False,False,False,True,False,False,False,False,False,True
1,2022-01-01,0.011707,0.001093,False,1709,102.0,570.0,2022,1,1,...,False,True,False,False,True,False,False,False,False,False
2,2022-01-01,0.007886,0.038150,False,900,147.0,991.0,2022,1,1,...,False,False,False,True,False,False,False,False,False,True
3,2022-01-01,0.038764,0.018900,False,1517,153.0,986.0,2022,1,1,...,False,True,False,False,True,False,False,False,False,False
4,2022-01-01,0.006143,0.013600,False,545,179.0,1062.0,2022,1,1,...,False,False,False,False,False,False,False,True,False,False


### Target encoding (Tail Number)

In [47]:
target_mean_encoding = (
    train_enc.groupby("Tail_Number")["Cancelled"].mean().to_dict()
)

train_enc["Tail_Number"] = train_enc["Tail_Number"].map(target_mean_encoding)
test_enc["Tail_Number"]  = test_enc["Tail_Number"].map(target_mean_encoding)

train_enc.head()

,FlightDate,Origin,Dest,Cancelled,CRSDepTime,CRSElapsedTime,Distance,Year,Quarter,Month,...,OriginState_Northeast,OriginState_South,OriginState_Unknown/Other,OriginState_West,DestState_Midwest,DestState_Non-continental US,DestState_Northeast,DestState_South,DestState_Unknown/Other,DestState_West
0,2022-01-01,0.007171,0.023493,False,1500,85.0,386.0,2022,1,1,...,False,False,False,True,False,False,False,False,False,True
1,2022-01-01,0.011707,0.001093,False,1709,102.0,570.0,2022,1,1,...,False,True,False,False,True,False,False,False,False,False
2,2022-01-01,0.007886,0.038150,False,900,147.0,991.0,2022,1,1,...,False,False,False,True,False,False,False,False,False,True
3,2022-01-01,0.038764,0.018900,False,1517,153.0,986.0,2022,1,1,...,False,True,False,False,True,False,False,False,False,False
4,2022-01-01,0.006143,0.013600,False,545,179.0,1062.0,2022,1,1,...,False,False,False,False,False,False,False,True,False,False


In [48]:
output_train_path = "../../data/prepared/encoding/flight_train_enc.csv"
output_test_path  = "../../data/prepared/encoding/flight_test_enc.csv"

train_enc.to_csv(output_train_path, index=False)
test_enc.to_csv(output_test_path, index=False)

print(f"Saved Encoding TRAIN dataset to: {output_train_path}")
print(f"Saved Encoding TEST dataset to:  {output_test_path}")

Saved Encoding TRAIN dataset to: ../../data/prepared/encoding/flight_train_enc.csv
Saved Encoding TEST dataset to:  ../../data/prepared/encoding/flight_test_enc.csv
